## Creating a Dictionary-based Sentiment Analyzer

Given the small corpus (reviews textual dataset) generated in task 1, the objective is to construct a dictionary-based sentiment analyser.
<br>
<br>
Some of the lessons taught during these tasks:
* Word and sentence tokenization 
* Review score classification 
* Insights surrounding score and review comparisons 
* Correlation analysis surrounding review groups 
* Accounting for negation within sentiment analyser



In [3]:
## Necessary imports needed for analysis 
import pandas as pd 
import numpy as np 
import os, pathlib

### Load small corpus dataset

In [4]:
# Move back into task1 repo where review (small) corpus is located
path = pathlib.Path().home()/'Desktop/nlp-map-project/task1-create-dataset'
try:
    path = os.chdir(path) 
except FileNotFoundError as a :
    print('Already in directory/folder, carry on!')

In [6]:
df = pd.read_csv('small_corpus.csv')